# 🍿 Content-Based Aanbevelingen

In dit notebook laten we snel zien hoe je een **content-based** aanbevelingssysteem kunt opzetten. Bij content-based filtering worden aanbevelingen gedaan op basis van de eigenschappen van items, in dit geval films. We gebruiken de kenmerken zoals `Genre`, `Actors`, `Plot`, `Title`, `Director`, en `Country` om de overeenkomst tussen films te berekenen. Deze aanpak is nuttig voor aanbevelingen waarbij de inhoud van een film overeenkomt met films die een gebruiker al leuk vindt.

### Hoe werkt het?
Het systeem berekent de overeenkomst tussen films door een **TF-IDF vectorizer** te gebruiken om woorden in de kenmerken te transformeren naar numerieke representaties, gevolgd door het berekenen van **cosine similarity** om de overeenkomst tussen films te meten. Dit betekent dat films met vergelijkbare genres, acteurs of verhaallijnen waarschijnlijk aan elkaar worden aanbevolen.

### Functie: `get_content_based_recommendations`
De functie `get_content_based_recommendations` maakt aanbevelingen voor een specifieke film gebaseerd op de inhoudskenmerken. Hier is een stap-voor-stap uitleg van hoe de functie werkt:

1. **Combineer Kenmerken**: De functie voegt `Plot`, `Title`, `Genre`, `Director`, en `Country` samen tot één enkele tekstuele feature. Dit maakt het eenvoudiger om overeenkomsten tussen films te berekenen.
2. **TF-IDF Vectorisatie**: Een TF-IDF vectorizer zet deze gecombineerde kenmerken om naar een numerieke representatie, waarbij veelvoorkomende woorden minder gewicht krijgen.
3. **Cosine Similarity Berekenen**: De functie berekent de cosine similarity tussen de geselecteerde film en alle andere films in de dataset.
4. **Sorteren op Overeenkomst**: De films worden gesorteerd op basis van hun overeenkomst met de geselecteerde film, en de top N meest vergelijkbare films worden geselecteerd (behalve de gekozen film zelf).
5. **Aanbevelingen Teruggeven**: Uiteindelijk geeft de functie een DataFrame terug met aanbevolen films.

### Voorbeeld Code
Hieronder zie je de code die de content-based aanbevelingen maakt:


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

df_movies = pd.read_csv('data/movies.csv')


def get_content_based_recommendations(movie_id, df, top_n=6):
    # Combine 'Genre' and 'Actors' into a single feature for similarity calculation
    df['combined_features'] = df['Plot'].fillna('') + ' ' + df['Title'].fillna('') + ' ' + df['Genre'].fillna('') + ' ' + df['Director'].fillna('') + ' ' + df['Country'].fillna('')
    
    # Initialize the TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    
    # Fit and transform the combined features
    tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])
    
    # Calculate cosine similarity matrix
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Get the index of the selected movie based on its movieId
    idx = df.index[df['movieId'] == movie_id].tolist()[0]
    
    # Get similarity scores for this movie, sorted by similarity
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    
    # Get indices of the most similar movies (excluding the selected movie itself)
    similar_movie_indices = [i[0] for i in similarity_scores[1:top_n + 1]]
    
    # Return DataFrame of recommended movies
    return df.iloc[similar_movie_indices]

In [ ]:
df_movies[df_movies['movieId'] == 3114]

In [ ]:
get_content_based_recommendations(3114, df_movies, top_n=6)